# Evaluation der Iterationsplots 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import sys, os, re
os.chdir("..")
import src.parse as parse
from src.plot import plot_all, automatic_plotting

In [6]:
dfs = {}
methods = ["SPPRC","cyc2","ng8","ng20"]
columns = ['instance','terminated']
finished_columns = ["SPPRC","cyc2","ng8"]
data = pd.DataFrame(columns=columns + finished_columns)
pattern = r"(X-n(\d+)-k(\d+)).log"
for i, log in enumerate((log for log in os.listdir("output_iterations"))):
    ID = log.strip(".log")
    instance, n, K = re.search(pattern, log).groups()
    log = f"output_iterations/{log}"
    finished = parse.log_finished(log)
    df = parse.parse_output(log)
    sol_val = df[df.method == "ng20"]['lower_bound'].iloc[-1]
    dfs[ID] = df
    if finished:
        row = []
        for method in finished_columns:
            lb = df[df.method == method]['lower_bound'].iloc[-1]
            row.append(lb/sol_val)
    if finished:
        data = pd.concat((data,pd.DataFrame([[instance, finished, *row]], columns=columns + finished_columns)))
    else:
        data = pd.concat((data,pd.DataFrame([[instance, finished]], columns=columns)))
data = data.sort_values("instance")
data = data.set_index(data.instance)[["SPPRC","cyc2","ng8"]]
data[data<0] = 0
data = data.sort_values("instance")
# data.to_latex("plots/table_iterations.tex", float_format="{:.2f}".format,na_rep="-")
# data.style

In [8]:
df = dfs["X-n120-k6"]
df

,method,python time,total time,propagated dominance time,unpropagated dominance time,pricing_success,upper_bound,lower_bound,abort_early,"num_paths,"
0,ESPPRC_heur,0.1,0.0,0.0,0.0,0,13267.9563,-10519.9380,True,1194
1,ESPPRC_heur,0.1,0.0,0.0,0.0,0,13267.9563,-27058.9475,True,1324
2,ESPPRC_heur,0.1,0.0,0.0,0.0,0,13267.9563,-26339.8962,True,1318
3,ESPPRC_heur,0.1,0.0,0.0,0.0,0,13267.9563,-49304.8277,True,1341
4,ESPPRC_heur,0.1,0.0,0.0,0.0,0,13267.9563,-36802.5742,True,1390
...,...,...,...,...,...,...,...,...,...,...
1030,SPPRC,-0.0,0.1,0.0,0.0,1,12954.1892,3894.9631,0,922
1031,ng20,0.1,3972.2,3771.0,193.3,1,12954.1892,12954.1892,0,0
1032,ng8,0.0,2.0,1.6,0.2,1,12954.1892,12726.8441,0,40
1033,cyc2,0.0,0.2,0.0,0.0,1,12954.1892,8750.4127,0,3375


In [42]:
data.mean()

SPPRC    0.516671
cyc2     0.831887
ng8      0.952441
dtype: float64